#### -- ANT COLONY OPTIMIZATION --


- Ant Colony Optimization

	1. Inspiración en las hormigas: ACO se basa en la observación de cómo las hormigas encuentran el camino más corto entre su colonia y una fuente de comida. Las hormigas dejan feromonas en el suelo mientras se mueven. Estas feromonas atraen a otras hormigas y, con el tiempo, se forma un camino óptimo hacia la comida.

	2. Modelo de colonia de hormigas: En ACO, se simula una colonia de hormigas artificiales. Cada hormiga se mueve por un espacio de soluciones, tratando de encontrar una solución óptima al problema. Cada solución es representada como una secuencia de decisiones.

	3. Construcción de soluciones: Las hormigas construyen soluciones iterativamente. En cada paso, una hormiga elige su próxima decisión basada en dos factores principales: la información de feromonas y una heurística. La información de feromonas representa la calidad de una decisión en particular, y la heurística proporciona información adicional sobre la solución.

	4. Actualización de feromonas: Después de que todas las hormigas hayan construido una solución, se actualizan las feromonas en el camino seguido por cada hormiga. Las feromonas evaporan con el tiempo para evitar que las soluciones subóptimas acumulen demasiada feromona.

	5. Exploración vs. Explotación: ACO equilibra la exploración (buscar nuevas soluciones) y la explotación (mejorar soluciones conocidas) mediante el uso de parámetros que controlan la influencia de las feromonas y la heurística en la toma de decisiones de las hormigas.

	6. Criterio de parada: El algoritmo ACO continúa iterando hasta que se cumple un criterio de parada, que puede ser un número fijo de iteraciones, un límite de tiempo o una mejora en la solución actual.

	7. Solución óptima: Con el tiempo, las feromonas se acumulan en las mejores soluciones, lo que guía a las hormigas hacia soluciones cada vez más óptimas. Finalmente, la mejor solución encontrada por las hormigas se considera la solución óptima o una aproximación cercana a ella.

		*** ACO es especialmente eficaz en problemas donde es difícil encontrar una solución óptima de manera exhaustiva debido a la alta complejidad combinatoria. La capacidad de ACO para explorar múltiples soluciones y converger hacia soluciones de alta calidad lo hace valioso en una variedad de aplicaciones, desde la optimización de rutas logísticas hasta el diseño de circuitos electrónicos. ***


### IMPLEMENTACIÓN 1

In [122]:
#=======
#ACO
#=======

import numpy as np
import pandas as pd

# Definimos nuestros datos
# 5 almacenes con su respectiva capacidad y distancia a cada uno

data = {
    'almacen': ['A', 'B', 'C', 'D', 'E'],
    'capacidad': [100, 150, 200, 250, 300],
    'distancia': [10, 20, 30, 40, 50],
}

df = pd.DataFrame(data).set_index('almacen')

In [ ]:
# Definimos la función objetivo
# Esta función calcula el coste de la asignación de la cantidad de cada material a un almacen
# Este coste será la suma de material multiplicado por la distancia al almacen

def objetivo(asignacion, df):
    distancia_total = 0
    for almacen, cantidad in asignacion.items():
        distancia_total += sum(cantidad.values()) * df.loc[almacen, 'distancia']
    return distancia_total

In [113]:
# Definimos los parámetros
# Número de hormigas a utilizar
# probabilidad de desaparición de feromonas
# intensidad de las feromonas

n_homigas = 200
evaporizacion_f = 0.1
cantidad_f = 1.0

# Feromonas para cada almacen y cada tipo de material

feromonas = np.ones((5, 3))

for it in range(1000):
    # Soluciones/caminos seguidos en cada iteración
    sol = []
    # Iteramos sobre cada hormiga
    for _ in range(n_homigas):
        # Cada hormiga lleva 100 de cada material
        material = {'Tipo1': 100, 'Tipo2': 100, 'Tipo3': 100}
        # Inicializamos la asignación de cada almacen a 0
        asignacion = {almacen: {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0} for almacen in df.index}
        # Iteramos tres veces
        for _ in range(3):
            # Calculamos la probabilidad de asignar un material a cada almacen 
            # La probabilidad es la proporcion de feromonas de cese material en cada almacen
            prob = feromonas.sum(axis=1) / (feromonas.sum(axis=1).sum() + 1e-10)
            # Elegimos de forma aleatoria un almacen asignado en función de la probabilidad
            selec_almacen = np.random.choice(df.index, p=prob)

            # Iteramos sobre cada tipo de material
            for tipo, cantidad in material.items():
                # Cantidad de material de un tipo asignada a este almacen = minimo entre lo que lleva la hormiga y el espacio restante
                asignado = min(cantidad, df.loc[selec_almacen, 'capacidad'] - sum(asignacion[selec_almacen].values()))
                # Sumamos la cantidad de material de este tipo a lo almacenado en este almacen
                asignacion[selec_almacen][tipo] += asignado
                # Se lo quitamos a la hormiga
                material[tipo] -= asignado
        # Guardamos las asignaciones elegidas junto con su coste asociado
        sol.append((asignacion, objetivo(asignacion, df)))

    # Actualizamos el número de feromonas con la evaporización
    feromonas *= (1 - evaporizacion_f)
    # Iteramos sobre las asignaciones realizadas hasta ahora y su coste (solucion)
    for asignacion, distancia in sol:
        # Iteramos sobre cada almacen
        for almacen in asignacion.keys():
            # Actualizamos las feromonas asociadas a cada almacen sumando la relación entre 
            # el número de feromonas portadas y el coste de la solución
            feromonas[df.index.get_loc(almacen), :] += cantidad_f / distancia

# Elegimos la asignación con menor coste
mejor_asignacion, mejor_distancia = min(sol, key=lambda x: x[1])
print('Mejor asignacion:', mejor_asignacion)
print('Distancia total:', mejor_distancia)


Assegnazione Migliore: {'A': {'Tipo1': 100, 'Tipo2': 0, 'Tipo3': 0}, 'B': {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}, 'C': {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}, 'D': {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}, 'E': {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}}
Distanza Totale: 1000


### IMPLEMENTACIÓN 2

In [1]:
import numpy as np
import random

# Datos del problema - Número de almacenes, capacidad de cada uno y matriz de distancias que los separan
num_almacenes = 5
capacidades = [100, 150, 200, 250, 300]
distancias = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 10, 20],
    [30, 25, 10, 0, 15],
    [40, 35, 20, 15, 0]
]

In [5]:
# Función objetivo: Calcula la distancia total de la asignación actual
# La función de coste se calcula como la contracción entre la matriz de distancias y de asignaciones
# El coste asociado a cada viaje sera la cantidad de material * la distancia del viaje
def calcular_distancia(asignacion):
    distancia_total = 0
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            distancia_total += distancias[i][j] * asignacion[i][j]
    return distancia_total

# Genera una solución inicial aleatoria
def generar_solucion_inicial():
    asignacion = [[0] * num_almacenes for _ in range(num_almacenes)]
    for i in range(num_almacenes):
        capacidad_restante = capacidades[i]
        for j in range(num_almacenes):
            if i != j:
                cantidad_asignada = random.randint(0, capacidad_restante)
                asignacion[i][j] = cantidad_asignada
                capacidad_restante -= cantidad_asignada
    return asignacion

# Construye una solución basada en feromonas y heurísticas
def construir_solucion():
    # Matriz de asignaciones
    asignacion = [[0] * num_almacenes for _ in range(num_almacenes)]
    # Iteramos sobre los amacenes copiando su capacidad restante
    for _ in range(num_almacenes):
        capacidad_restante = capacidades.copy()
        # Volvemos a iterar
        for i in range(num_almacenes):
            # Para cada almacen seleccionamosel respectivo destino de materiales
            j = seleccionar_siguiente_almacen(i, capacidad_restante, intensidad_feromona)
            # Seleccionamos la cantidad de material enviada de forma aleatoria
            cantidad_asignada = random.randint(0, capacidad_restante[j])
            asignacion[i][j] = cantidad_asignada
            capacidad_restante[j] -= cantidad_asignada
    return asignacion

In [4]:
# Selecciona el siguiente almacén basado en feromonas y heurísticas
def seleccionar_siguiente_almacen(almacen_actual, capacidad_restante, intensidad_feromona):
    # Inicializamos probabilidades a 0
    probabilidad = [0.0] * num_almacenes
    total_probabilidad = 0.0
    # Iteramos sobre los almacenes
    for j in range(num_almacenes):
        if almacen_actual != j and capacidad_restante[j] > 0:
            # La probabilidad de elegir este almacen como el siguiente es proporcional al número de feromonas
            # en su arista elevado a la intensidad dividido entre su distancia de separación
            probabilidad[j] = (feromonas[almacen_actual][j] ** intensidad_feromona) * (1.0 / distancias[almacen_actual][j])
            total_probabilidad += probabilidad[j]

    # Calculamos la probabilidad real (normalizada) y devolvemos asignaciones aleatorias
    # en función de la probabilidad para cada almacén
    if total_probabilidad > 0:
        probabilidad = [p / total_probabilidad for p in probabilidad]
        return np.random.choice(num_almacenes, 1, p=probabilidad)[0]
    else:
        return np.argmax(capacidad_restante)
    # Si no hay probabilidad porque no hay feromonas devolvemos el almacén con mayor capacidad restante

# Actualiza las feromonas en función de las asignaciones realizadas
def actualizar_feromonas(asignacion, mejor_asignacion, evaporacion_feromona, intensidad_feromona):
    # Iteramos sobre la matriz de asignaciones
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            # Disminuimos todas las feromonas asignadas en cada conexión
            feromonas[i][j] *= (1.0 - evaporacion_feromona)
            # En el caso de haber elegido ciertos caminos actualizamos las feromonas
            # de los mismos sumando una cantidad proporcional a la intensidad e inversa a la distancia total de la
            # mejor asignación
            if asignacion[i][j] > 0:
                feromonas[i][j] += intensidad_feromona / calcular_distancia(mejor_asignacion)

In [6]:

# Parámetros del algoritmo ACO
num_hormigas = 50
evaporacion_feromona = 0.1
intensidad_feromona = 1.0
max_iteraciones = 500

# Inicialización de feromonas
feromonas = np.ones((num_almacenes, num_almacenes))

# Algoritmo ACO
# Generamos una primera solución aleatoria en función de la capacidad = mejor asigación
mejor_asignacion_global = generar_solucion_inicial()
# Calcualmos la distancia total de la primera solución
mejor_distancia_global = calcular_distancia(mejor_asignacion_global)
# Iteramos n veces == criterio de parada
for _ in range(max_iteraciones):
    # Lista de soluciones para esta iteración
    mejores_asignaciones_locales = []
    mejores_distancias_locales = []
    # Iteramos sobre todas las hormigas
    for _ in range(num_hormigas):
        # Construimos una solución con cada una = matriz de asignaciones
        # En función de las feromonas presentes en cada conexión y la capacidad restante
        asignacion_hormiga = construir_solucion()
        # Obtenemos el coste para esta solución
        distancia_hormiga = calcular_distancia(asignacion_hormiga)
        # La comparamos con la mejor solución y sustituimos
        if distancia_hormiga < mejor_distancia_global:
            mejor_asignacion_global = asignacion_hormiga
            mejor_distancia_global = distancia_hormiga
        # Guardamos todas las soluciones locales
        mejores_asignaciones_locales.append(asignacion_hormiga)
        mejores_distancias_locales.append(distancia_hormiga)
    # Al final de nuestra iteración actualizamos nuestras feromonas para las siguientes
    # hormigas en función de las asignaciones locales realizadas y la mejor asignación global
    for i in range(num_hormigas):
        actualizar_feromonas(mejores_asignaciones_locales[i], mejor_asignacion_global,evaporacion_feromona, intensidad_feromona)

# Mostrar resultados
print('Mejor Asignación Encontrada:')
for fila in mejor_asignacion_global:
    print(fila)
print('Distancia Total de la Mejor Asignación:', mejor_distancia_global)

Mejor Asignación Encontrada:
[0, 21, 0, 0, 0]
[3, 0, 0, 0, 0]
[12, 0, 0, 74, 0]
[0, 0, 89, 0, 40]
[0, 0, 2, 11, 0]
Distancia Total de la Mejor Asignación: 2915


#### -- GENETIC ALGORITHM OPTIMIZATION --

### IMPLEMENTACIÓN 1

In [ ]:
import numpy as np
import pandas as pd
import random

# Datos del problema
data = {
    'almacen': ['A', 'B', 'C', 'D', 'E'],
    'capacidad': [100, 150, 200, 250, 300],
    'distancia': [10, 20, 30, 40, 50],
}

df = pd.DataFrame(data).set_index('almacen')

In [9]:
# Función objetivo/coste
# Calcula el coste de una asignación como el producto 
# de la cantidad de material por la distancia
def objetivo(asignacion, df):
    distancia_total = 0
    for almacen, cantidad in asignacion.items():
        distancia_total += sum(cantidad.values()) * df.loc[almacen, 'distancia']
    return distancia_total

# Inicialización de la población
# Función para generar un individuo = una solución
# Cada solución asigna una cantidad de material de cada tipo a cada almacén de forma aleatoria
# la cantidad también es aleatoria en función de la capacidad del almacén
def generar_individuo():
    individuo = {}
    for almacen in df.index:
        tipos = {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}
        for _ in range(3):
            tipo = random.choice(list(tipos.keys()))
            cantidad = random.randint(0, min(100, df.loc[almacen, 'capacidad']))
            tipos[tipo] = cantidad
        individuo[almacen] = tipos
    return individuo

In [ ]:
# Función de selección (Torneo binario)
# Realizamos una selección de individuos
def seleccion(poblacion, tamano_poblacion):
    seleccionados = []
    for _ in range(tamano_poblacion):
        # Elegimos el mejor individuo de cada 5 generados de forma aleatoria en función de su fitness
        subpoblacion = random.sample(poblacion, 5)  # Selección de 5 individuos al azar
        mejor_individuo = min(subpoblacion, key=lambda x: objetivo(x, df))
        seleccionados.append(mejor_individuo)
    return seleccionados

# Función de cruce (Crossover)
# Función que cruza los genes de dos individuos
def cruce(padre1, padre2):
    hijo = {}
    # Se recorre la asignación de materiales a cada almacén
    for almacen in df.index:
        tipos = {}
        for tipo in ['Tipo1', 'Tipo2', 'Tipo3']:
            # Se eligen de forma aleatoria asignaciones de uno y otro respectivamente
            tipo_padre = random.choice([padre1, padre2])[almacen][tipo]
            tipos[tipo] = tipo_padre
        hijo[almacen] = tipos
    # Devolvemos la nueva solución cruzada
    return hijo

# Función de mutación
# Función que altera de forma aleatoria los genes de un individuo
def mutacion(individuo, probabilidad_mutacion):
    # Recorremos las asignaciones y transformamos cada gen en funciónd de la probabilidad
    for almacen in df.index:
        for tipo in ['Tipo1', 'Tipo2', 'Tipo3']:
            if random.random() < probabilidad_mutacion:
                individuo[almacen][tipo] = random.randint(0, min(100, df.loc[almacen, 'capacidad']))
    return individuo

In [69]:

# Parámetros del algoritmo genético
# Población = número de individuos utilizados en cada generación/iteración
tamano_poblacion = 50
# Probabilidad de variar un gen de forma aleatoria
probabilidad_mutacion = 0.1
# Generaciones = iteraciones
num_generaciones = 100

# Generamos una población inicial de tamaño fijo = soluciones aleatorias

poblacion = [generar_individuo() for _ in range(tamano_poblacion)]

# Algoritmo genético
# Iteramos sobre cada generación de individuos
for generacion in range(num_generaciones):
    # Evaluación de la población = obtenemos el finess de todas las soluciones generadas
    evaluaciones = [(individuo, objetivo(individuo, df)) for individuo in poblacion]

    # Selección = fitramos el mejor 20% de la población
    poblacion = seleccion(poblacion, tamano_poblacion)

    # Cruce = Generamos nuevas soluciones a partir de combinaciones de la actuales
    descendientes = []
    for _ in range(tamano_poblacion // 2):
        # Elegimos padres y madres de forma aleatoria, podría ser por ranking
        padre1 = random.choice(poblacion)
        padre2 = random.choice(poblacion)
        # Generamos dos hijos y los guardamos
        hijo1 = cruce(padre1, padre2)
        hijo2 = cruce(padre1, padre2)
        # Guardamos nuestra nueva población
        descendientes.extend([hijo1, hijo2])

    # Mutación
    # Transformamos de forma aleatoria los genes de la nueva población
    poblacion = [mutacion(individuo, probabilidad_mutacion) for individuo in descendientes]

# Encuentra el mejor individuo de la población final en función de su fitness
mejor_asignacion, mejor_distancia = min(evaluaciones, key=lambda x: x[1])
print('Asignación Óptima:', mejor_asignacion)
print('Distancia Total:', mejor_distancia)


Asignación Óptima: {'A': {'Tipo1': 4, 'Tipo2': 2, 'Tipo3': 1}, 'B': {'Tipo1': 0, 'Tipo2': 1, 'Tipo3': 1}, 'C': {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 3}, 'D': {'Tipo1': 0, 'Tipo2': 4, 'Tipo3': 0}, 'E': {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}}
Distancia Total: 360


### IMPLEMENTACIÓN 2

In [124]:
import numpy as np
import random

# Datos del problema (debes ajustar estos datos según tu instancia específica)
num_almacenes = 5
capacidades = [100, 150, 200, 250, 300]
distancias = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 10, 20],
    [30, 25, 10, 0, 15],
    [40, 35, 20, 15, 0]
]

# Función objetivo: Calcula la distancia total de la asignación actual
def calcular_distancia(asignacion):
    distancia_total = 0
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            distancia_total += distancias[i][j] * asignacion[i][j]
    return distancia_total

# Genera una solución inicial aleatoria
def generar_solucion_inicial():
    asignacion = [[0] * num_almacenes for _ in range(num_almacenes)]
    for i in range(num_almacenes):
        capacidad_restante = capacidades[i]
        for j in range(num_almacenes):
            if i != j:
                cantidad_asignada = random.randint(0, capacidad_restante)
                asignacion[i][j] = cantidad_asignada
                capacidad_restante -= cantidad_asignada
    return asignacion

# Función para crear una población inicial de soluciones aleatorias
def crear_poblacion_inicial(tamano_poblacion):
    poblacion = []
    for _ in range(tamano_poblacion):
        solucion = generar_solucion_inicial()
        poblacion.append(solucion)
    return poblacion

# Función de selección de padres (selección de torneo)
def seleccion_de_padres(poblacion, tamano_torneo):
    torneo = random.sample(poblacion, tamano_torneo)
    mejor_solucion = min(torneo, key=lambda x: calcular_distancia(x))
    return mejor_solucion

# Función de cruce (intercambio de asignaciones)
def cruce(padre1, padre2):
    hijo = [[0] * num_almacenes for _ in range(num_almacenes)]
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            if i != j:
                cantidad_hijo = random.choice([padre1[i][j], padre2[i][j]])
                hijo[i][j] = cantidad_hijo
    return hijo

# Función de mutación (cambio aleatorio en las asignaciones)
def mutacion(solucion, probabilidad_mutacion):
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            if i != j and random.random() < probabilidad_mutacion:
                cantidad_nueva = random.randint(0, capacidades[i])
                solucion[i][j] = cantidad_nueva
    return solucion

# Algoritmo Genético
def algoritmo_genetico(tamano_poblacion, tamano_torneo, probabilidad_cruce, probabilidad_mutacion, num_generaciones):
    poblacion = crear_poblacion_inicial(tamano_poblacion)

    for _ in range(num_generaciones):
        nueva_poblacion = []

        while len(nueva_poblacion) < tamano_poblacion:
            padre1 = seleccion_de_padres(poblacion, tamano_torneo)
            padre2 = seleccion_de_padres(poblacion, tamano_torneo)

            if random.random() < probabilidad_cruce:
                hijo = cruce(padre1, padre2)
            else:
                hijo = padre1  # Sin cruce, se selecciona uno de los padres

            hijo = mutacion(hijo, probabilidad_mutacion)
            nueva_poblacion.append(hijo)

        poblacion = nueva_poblacion

    mejor_solucion = min(poblacion, key=lambda x: calcular_distancia(x))
    mejor_distancia = calcular_distancia(mejor_solucion)
    return mejor_solucion, mejor_distancia

# Configuración del algoritmo genético
tamano_poblacion = 200
tamano_torneo = 20
probabilidad_cruce = 0.5
probabilidad_mutacion = 0.1
num_generaciones = 200

# Ejecutar algoritmo genético
mejor_solucion, mejor_distancia = algoritmo_genetico(tamano_poblacion, tamano_torneo, probabilidad_cruce, probabilidad_mutacion, num_generaciones)

# Mostrar resultados
print('Mejor Asignación Encontrada:')
for fila in mejor_solucion:
    print(fila)
print('Distancia Total de la Mejor Asignación:', mejor_distancia)


Mejor Asignación Encontrada:
[0, 40, 10, 28, 4]
[19, 0, 15, 11, 38]
[6, 0, 0, 86, 5]
[13, 0, 87, 0, 7]
[3, 9, 9, 5, 0]
Distancia Total de la Mejor Asignación: 6755


### -- TABU SEARCH --

### IMPLEMENTACIÓN 1

In [23]:
import numpy as np
import pandas as pd
import random

# Datos del problema
data = {
    'almacen': ['A', 'B', 'C', 'D', 'E'],
    'capacidad': [100, 150, 200, 250, 300],
    'distancia': [10, 20, 30, 40, 50],
}

df = pd.DataFrame(data).set_index('almacen')

In [24]:
# Función objetivo
def objetivo(asignacion, df):
    distancia_total = 0
    for almacen, cantidad in asignacion.items():
        distancia_total += sum(cantidad.values()) * df.loc[almacen, 'distancia']
    return distancia_total

# Generación de una solución inicial aleatoria
def generar_solucion_inicial(df):
    solucion = {}
    for almacen in df.index:
        tipos = {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}
        for _ in range(3):
            tipo = random.choice(list(tipos.keys()))
            cantidad = random.randint(0, min(100, df.loc[almacen, 'capacidad']))
            tipos[tipo] = cantidad
        solucion[almacen] = tipos
    return solucion

In [25]:
# Generación de vecindario (vecinos) de una solución
def generar_vecindario(solucion_actual, df):
    vecindario = []
    # Recorremos todos los almacenes
    for almacen in df.index:
        for otro_almacen in df.index:
            if almacen != otro_almacen:
                vecino = solucion_actual.copy()
                # Los vecinos serán copias de la solución actual pero moviendo los materiales
                # asignados a cada almacén hasta otro distinto
                for tipo in ['Tipo1', 'Tipo2', 'Tipo3']:
                    cantidad_disponible = df.loc[otro_almacen, 'capacidad'] - sum(solucion_actual[otro_almacen].values())
                    cantidad_a_mover = min(cantidad_disponible, solucion_actual[almacen][tipo])
                    vecino[almacen][tipo] -= cantidad_a_mover
                    vecino[otro_almacen][tipo] += cantidad_a_mover
                vecindario.append(vecino)
    return vecindario

In [53]:
# Parámetros del algoritmo Tabu Search
# Tamaño de la lista de soluciones descartadas
tamano_lista_tabu = 10
num_iteraciones = 10000

# Algoritmo de Tabu Search
def tabu_search(df, tamano_lista_tabu, num_iteraciones):
    # Obtenemos una solución inicial aleatoria y su coste asociado
    mejor_solucion = generar_solucion_inicial(df)
    mejor_distancia = objetivo(mejor_solucion, df)
    # Inicializamos la lista tabu == Lista dinámica de vecinos encontrados en cada iteración
    lista_tabu = []

    for _ in range(num_iteraciones):
        # Dada una solucion generamos un vecindario = soluciones similares
        # las soluciones se obtienen de mover los materiales entre almacenes
        
        vecindario = generar_vecindario(mejor_solucion, df)
        # Ordenamos las soluciones por coste
        vecindario.sort(key=lambda x: objetivo(x, df))
        # Evaluamos el conjunto de vecinos
        for vecino in vecindario:
            # Evaluamos si el vecino esta incluido o si es mejor solucion
            if vecino not in lista_tabu or objetivo(vecino, df) < mejor_distancia:
                # Sustituimos la solucion global
                mejor_solucion = vecino
                mejor_distancia = objetivo(vecino, df)
                # Lo guardamos en la lista tabu
                lista_tabu.append(vecino)
                # Conservamos la longitud de la lista tabu
                if len(lista_tabu) > tamano_lista_tabu:
                    lista_tabu.pop(0)  # Eliminar el elemento más antiguo de la lista Tabú
                break

    return mejor_solucion, mejor_distancia

# Ejecutar Tabu Search
mejor_asignacion, mejor_distancia = tabu_search(df, tamano_lista_tabu, num_iteraciones)

# Mostrar resultados
print('Asignación Óptima (Tabu Search):', mejor_asignacion)
print('Distancia Total (Tabu Search):', mejor_distancia)


Asignación Óptima (Tabu Search): {'A': {'Tipo1': 74, 'Tipo2': 26, 'Tipo3': 0}, 'B': {'Tipo1': 9, 'Tipo2': 104, 'Tipo3': 37}, 'C': {'Tipo1': 66, 'Tipo2': 74, 'Tipo3': 59}, 'D': {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}, 'E': {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}}
Distancia Total (Tabu Search): 9970


# IMPLEMENTACIÓN 2

In [ ]:
import numpy as np
import random

# Datos del problema (ajusta estos datos según tu instancia específica)
num_almacenes = 5
capacidades = [100, 150, 200, 250, 300]
distancias = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 10, 20],
    [30, 25, 10, 0, 15],
    [40, 35, 20, 15, 0]
]

In [ ]:
# Función objetivo: Calcula la distancia total de la asignación actual
def calcular_distancia(asignacion):
    distancia_total = 0
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            distancia_total += distancias[i][j] * asignacion[i][j]
    return distancia_total

# Genera una solución inicial aleatoria
def generar_solucion_inicial():
    asignacion = [[0] * num_almacenes for _ in range(num_almacenes)]
    for i in range(num_almacenes):
        capacidad_restante = capacidades[i]
        for j in range(num_almacenes):
            if i != j:
                cantidad_asignada = random.randint(0, capacidad_restante)
                asignacion[i][j] = cantidad_asignada
                capacidad_restante -= cantidad_asignada
    return asignacion

In [ ]:
# Búsqueda Tabú: Encuentra una mejor solución vecina mediante intercambio de recursos
def busqueda_tabu(asignacion_actual, max_iteraciones, tamano_lista_tabu):
    mejor_asignacion = asignacion_actual
    mejor_distancia = calcular_distancia(asignacion_actual)
    
    iteracion_actual = 0
    lista_tabu = []

    while iteracion_actual < max_iteraciones:
        # Generamos 4 índices de almacen aleatorios == Una combinación
        i, j, k, l = random.sample(range(num_almacenes), 4)
        # Realizamos un intercambio de recursos asignados entre dos almacenes diferentes
        # en función de la capacidad máxima y si el intercambio no está presente en la lista tabu
        if i != j and k != l and (i, j, k, l) not in lista_tabu:
            vecina_asignacion = [fila[:] for fila in asignacion_actual]
            cantidad_ij = min(vecina_asignacion[i][j], capacidades[k] - vecina_asignacion[k][l])
            cantidad_kl = min(vecina_asignacion[k][l], capacidades[i] - vecina_asignacion[i][j])
            vecina_asignacion[i][j] -= cantidad_ij
            vecina_asignacion[k][l] += cantidad_ij
            vecina_asignacion[k][l] -= cantidad_kl
            vecina_asignacion[i][j] += cantidad_kl

            distancia_vecina = calcular_distancia(vecina_asignacion)
            
            if distancia_vecina < mejor_distancia:
                mejor_asignacion = vecina_asignacion
                mejor_distancia = distancia_vecina

            # Guardamos esta combinación en la lista tabu
            lista_tabu.append((i, j, k, l))
            if len(lista_tabu) > tamano_lista_tabu:
                lista_tabu.pop(0)

            iteracion_actual += 1

    return mejor_asignacion, mejor_distancia

# Configuración de Búsqueda Tabú
max_iteraciones = 10000
tamano_lista_tabu = 10

# Ejecutar Búsqueda Tabú
solucion_inicial = generar_solucion_inicial()
mejor_solucion, mejor_distancia = busqueda_tabu(solucion_inicial, max_iteraciones, tamano_lista_tabu)

# Mostrar resultados
print('Mejor Asignación Encontrada:')
for fila in mejor_solucion:
    print(fila)
print('Distancia Total de la Mejor Asignación:', mejor_distancia)


### -- SIMULATED ANNEALING --

### IMPLEMENTACIÓN 1

In [73]:
import numpy as np
import pandas as pd
import random
import math

# Datos del problema
data = {
    'almacen': ['A', 'B', 'C', 'D', 'E'],
    'capacidad': [100, 150, 200, 250, 300],
    'distancia': [10, 20, 30, 40, 50],
}

df = pd.DataFrame(data).set_index('almacen')

# Parámetros del algoritmo Simulated Annealing
temperatura_inicial = 200
temperatura_final = 0.05
factor_enfriamiento = 0.1
num_iteraciones_por_temperatura = 500

# Función objetivo
def objetivo(asignacion, df):
    distancia_total = 0
    for almacen, cantidad in asignacion.items():
        distancia_total += sum(cantidad.values()) * df.loc[almacen, 'distancia']
    return distancia_total

# Generación de una solución inicial aleatoria
def generar_solucion_inicial(df):
    solucion = {}
    for almacen in df.index:
        tipos = {'Tipo1': 0, 'Tipo2': 0, 'Tipo3': 0}
        for _ in range(3):
            tipo = random.choice(list(tipos.keys()))
            cantidad = random.randint(0, min(100, df.loc[almacen, 'capacidad']))
            tipos[tipo] = cantidad
        solucion[almacen] = tipos
    return solucion

# Generación de una solución vecina (vecino) mediante perturbación
def generar_solucion_vecina(solucion_actual, df):
    vecino = solucion_actual.copy()
    almacen = random.choice(df.index)
    otro_almacen = random.choice(df.index)
    
    if almacen != otro_almacen:
        tipo = random.choice(['Tipo1', 'Tipo2', 'Tipo3'])
        cantidad = random.randint(0, min(100, df.loc[otro_almacen, 'capacidad']))
        
        # Mover recursos de un almacén a otro
        vecino[almacen][tipo] -= cantidad
        vecino[otro_almacen][tipo] += cantidad
    
    return vecino

# Algoritmo Simulated Annealing
def simulated_annealing(df, temperatura_inicial, temperatura_final, factor_enfriamiento, num_iteraciones_por_temperatura):
    mejor_solucion = generar_solucion_inicial(df)
    mejor_distancia = objetivo(mejor_solucion, df)
    solucion_actual = mejor_solucion
    distancia_actual = mejor_distancia
    temperatura = temperatura_inicial

    while temperatura > temperatura_final:
        for _ in range(num_iteraciones_por_temperatura):
            vecino = generar_solucion_vecina(solucion_actual, df)
            distancia_vecino = objetivo(vecino, df)
            diferencia_distancia = distancia_vecino - distancia_actual

            if diferencia_distancia < 0 or random.random() < math.exp(-diferencia_distancia / temperatura):
                solucion_actual = vecino
                distancia_actual = distancia_vecino

                if distancia_actual < mejor_distancia:
                    mejor_solucion = solucion_actual
                    mejor_distancia = distancia_actual

        temperatura *= factor_enfriamiento

    return mejor_solucion, mejor_distancia

# Ejecutar Simulated Annealing
mejor_asignacion, mejor_distancia = simulated_annealing(df, temperatura_inicial, temperatura_final, factor_enfriamiento, num_iteraciones_por_temperatura)

# Mostrar resultados
print('Asignación Óptima (Simulated Annealing):', mejor_asignacion)
print('Distancia Total (Simulated Annealing):', mejor_distancia)


Asignación Óptima (Simulated Annealing): {'A': {'Tipo1': 1635, 'Tipo2': 549, 'Tipo3': -1957}, 'B': {'Tipo1': -956, 'Tipo2': -61, 'Tipo3': 1779}, 'C': {'Tipo1': 830, 'Tipo2': -277, 'Tipo3': -196}, 'D': {'Tipo1': -768, 'Tipo2': 1310, 'Tipo3': -149}, 'E': {'Tipo1': -604, 'Tipo2': -1211, 'Tipo3': 801}}
Distancia Total (Simulated Annealing): -6760


In [93]:
import numpy as np
import random
import math

# Datos del problema (ajusta estos datos según tu instancia específica)
num_almacenes = 5
capacidades = [100, 150, 200, 250, 300]
distancias = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 10, 20],
    [30, 25, 10, 0, 15],
    [40, 35, 20, 15, 0]
]

# Función objetivo: Calcula la distancia total de la asignación actual
def calcular_distancia(asignacion):
    distancia_total = 0
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            distancia_total += distancias[i][j] * asignacion[i][j]
    return distancia_total

# Genera una solución inicial aleatoria
def generar_solucion_inicial():
    asignacion = [[0] * num_almacenes for _ in range(num_almacenes)]
    for i in range(num_almacenes):
        capacidad_restante = capacidades[i]
        for j in range(num_almacenes):
            if i != j:
                cantidad_asignada = random.randint(0, capacidad_restante)
                asignacion[i][j] = cantidad_asignada
                capacidad_restante -= cantidad_asignada
    return asignacion

# Genera una solución vecina mediante intercambio de recursos
def generar_solucion_vecina(solucion_actual):
    i, j, k, l = random.sample(range(num_almacenes), 4)
    vecina = [fila[:] for fila in solucion_actual]
    
    cantidad_ij = min(vecina[i][j], capacidades[k] - vecina[k][l])
    cantidad_kl = min(vecina[k][l], capacidades[i] - vecina[i][j])
    
    vecina[i][j] -= cantidad_ij
    vecina[k][l] += cantidad_ij
    vecina[k][l] -= cantidad_kl
    vecina[i][j] += cantidad_kl
    
    return vecina

# Simulated Annealing
def simulated_annealing(temp_inicial, factor_enfriamiento, num_iteraciones):
    solucion_actual = generar_solucion_inicial()
    distancia_actual = calcular_distancia(solucion_actual)
    
    mejor_solucion = solucion_actual
    mejor_distancia = distancia_actual
    
    for iteracion in range(num_iteraciones):
        temperatura_actual = temp_inicial / (1 + factor_enfriamiento * iteracion)
        solucion_vecina = generar_solucion_vecina(solucion_actual)
        distancia_vecina = calcular_distancia(solucion_vecina)
        
        # Calcula la diferencia de distancia entre la solución vecina y la actual
        delta_distancia = distancia_vecina - distancia_actual
        
        # Acepta la solución vecina si es mejor o con probabilidad e^(-delta/T)
        if delta_distancia < 0 or random.random() < math.exp(-delta_distancia / temperatura_actual):
            solucion_actual = solucion_vecina
            distancia_actual = distancia_vecina
        
        # Actualiza la mejor solución encontrada
        if distancia_actual < mejor_distancia:
            mejor_solucion = solucion_actual
            mejor_distancia = distancia_actual
    
    return mejor_solucion, mejor_distancia

# Configuración de Simulated Annealing
temperatura_inicial = 1000
factor_enfriamiento = 0.1
num_iteraciones = 10000

# Ejecutar Simulated Annealing
mejor_solucion, mejor_distancia = simulated_annealing(temperatura_inicial, factor_enfriamiento, num_iteraciones)

# Mostrar resultados
print('Mejor Asignación Encontrada:')
for fila in mejor_solucion:
    print(fila)
print('Distancia Total de la Mejor Asignación:', mejor_distancia)


Mejor Asignación Encontrada:
[0, 52, 14, 8, 3]
[102, 0, 47, 9, 0]
[9, 82, 0, 150, 52]
[8, 4, 204, 0, 15]
[0, 4, 9, 26, 0]
Distancia Total de la Mejor Asignación: 10375


### -- A STAR --

In [48]:
import numpy as np
import pandas as pd
import heapq
from collections import namedtuple

# Definición de datos del problema
data = {
    'almacen': ['A', 'B', 'C', 'D', 'E'],
    'capacidad': [100, 150, 200, 250, 300],
    'distancia': [10, 20, 30, 40, 50],
}

df = pd.DataFrame(data).set_index('almacen')

# Clase para representar un estado del problema
State = namedtuple('State', ['almacen', 'asignacion', 'distancia'])

# Función para calcular el costo heurístico (distancia mínima a recorrer)
def heuristica(asignacion, df):
    distancia_total = 0
    for almacen, cantidad in asignacion.items():
        distancia_total += sum(cantidad.values()) * df.loc[almacen, 'distancia']
    return distancia_total

# Función para expandir los nodos vecinos desde un estado dado
def expandir_nodos(state, df):
    vecinos = []
    for almacen in df.index:
        for otro_almacen in df.index:
            if almacen != otro_almacen:
                vecino_asignacion = state.asignacion.copy()
                for tipo in ['Tipo1', 'Tipo2', 'Tipo3']:
                    cantidad_disponible = df.loc[otro_almacen, 'capacidad'] - sum(state.asignacion[otro_almacen].values())
                    cantidad_a_mover = min(cantidad_disponible, state.asignacion[almacen][tipo])
                    vecino_asignacion[almacen][tipo] -= cantidad_a_mover
                    vecino_asignacion[otro_almacen][tipo] += cantidad_a_mover
                vecino_distancia = state.distancia + df.loc[almacen, 'distancia'] * cantidad_a_mover
                vecinos.append(State(otro_almacen, vecino_asignacion, vecino_distancia))
    return vecinos

# Algoritmo A*
def a_estrella(df):
    estado_inicial = State('A', generar_solucion_inicial(df), 0)
    objetivo = State(None, {almacen: {'Tipo1': 100, 'Tipo2': 100, 'Tipo3': 100} for almacen in df.index}, float('inf'))

    cola_abierta = []
    heapq.heappush(cola_abierta, (0 + heuristica(estado_inicial.asignacion, df), estado_inicial))

    while cola_abierta:
        _, estado_actual = heapq.heappop(cola_abierta)

        if estado_actual == objetivo:
            return estado_actual.asignacion, estado_actual.distancia

        vecinos = expandir_nodos(estado_actual, df)
        for vecino in vecinos:
            f = vecino.distancia + heuristica(vecino.asignacion, df)
            heapq.heappush(cola_abierta, (f, vecino))

    return None

# Ejecutar A* y mostrar resultados
mejor_asignacion, mejor_distancia = a_estrella(df)
print('Asignación Óptima (A*):', mejor_asignacion)
print('Distancia Total (A*):', mejor_distancia)


KeyboardInterrupt: 

In [104]:
import numpy as np
import heapq

# Datos del problema (ajusta estos datos según tu instancia específica)
num_almacenes = 5
capacidades = [100, 150, 200, 250, 300]
distancias = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 10, 20],
    [30, 25, 10, 0, 15],
    [40, 35, 20, 15, 0]
]

# Función objetivo: Calcula la distancia total de la asignación actual
def calcular_distancia(asignacion):
    distancia_total = 0
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            distancia_total += distancias[i][j] * asignacion[i][j]
    return distancia_total

# Clase para representar un estado del problema
class EstadoAsignacion:
    def __init__(self, asignacion, distancia):
        self.asignacion = asignacion
        self.distancia = distancia

    def __lt__(self, otro):
        # Utilizado para la prioridad en la cola de prioridad
        return self.distancia < otro.distancia

# Función heurística para A* (puede ser nula en este caso)
def heuristica(asignacion_actual):
    return 0

# Expande un estado y genera estados sucesores
def expandir_estado(estado_actual):
    sucesores = []
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            if i != j:
                for k in range(num_almacenes):
                    for l in range(num_almacenes):
                        if k != l:
                            cantidad_ij = min(estado_actual.asignacion[i][j], capacidades[k] - estado_actual.asignacion[k][l])
                            cantidad_kl = min(estado_actual.asignacion[k][l], capacidades[i] - estado_actual.asignacion[i][j])

                            if cantidad_ij > 0 and cantidad_kl > 0:
                                nuevo_estado = [fila[:] for fila in estado_actual.asignacion]
                                nuevo_estado[i][j] -= cantidad_ij
                                nuevo_estado[k][l] += cantidad_ij
                                nuevo_estado[k][l] -= cantidad_kl
                                nuevo_estado[i][j] += cantidad_kl

                                nueva_distancia = calcular_distancia(nuevo_estado)
                                nuevo_estado_asignacion = EstadoAsignacion(nuevo_estado, nueva_distancia)
                                sucesores.append(nuevo_estado_asignacion)

    return sucesores

# Algoritmo A*
def a_estrella():
    estado_inicial = EstadoAsignacion(generar_solucion_inicial(), 0)
    cola_prioridad = [estado_inicial]
    visitados = set()

    while cola_prioridad:
        estado_actual = heapq.heappop(cola_prioridad)

        if calcular_distancia(estado_actual.asignacion) == estado_actual.distancia:
            return estado_actual.asignacion, estado_actual.distancia

        if tuple(map(tuple, estado_actual.asignacion)) in visitados:
            continue

        visitados.add(tuple(map(tuple, estado_actual.asignacion)))

        sucesores = expandir_estado(estado_actual)
        for sucesor in sucesores:
            heapq.heappush(cola_prioridad, sucesor)

    return None, float('inf')  # No se encontró solución

# Ejecutar A*
mejor_solucion, mejor_distancia = a_estrella()

# Mostrar resultados
print('Mejor Asignación Encontrada:')
for fila in mejor_solucion:
    print(fila)
print('Distancia Total de la Mejor Asignación:', mejor_distancia)

Mejor Asignación Encontrada:
[0, 61, 3, 5, 18]
[25, 0, 60, 31, 34]
[42, 0, 0, 72, 61]
[53, 96, 32, 0, 44]
[110, 2, 17, 145, 0]
Distancia Total de la Mejor Asignación: 19390


### -- LOCAL SEARCH --

### Implementación 2

In [ ]:
import numpy as np
import random

# Datos del problema (debes ajustar estos datos según tu instancia específica)
num_almacenes = 5
capacidades = [100, 150, 200, 250, 300]
distancias = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 10, 20],
    [30, 25, 10, 0, 15],
    [40, 35, 20, 15, 0]
]

In [ ]:
# Función objetivo: Calcula la distancia total de la asignación actual
def calcular_distancia(asignacion):
    distancia_total = 0
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            distancia_total += distancias[i][j] * asignacion[i][j]
    return distancia_total

# Genera una solución inicial aleatoria
def generar_solucion_inicial():
    asignacion = [[0] * num_almacenes for _ in range(num_almacenes)]
    for i in range(num_almacenes):
        capacidad_restante = capacidades[i]
        for j in range(num_almacenes):
            if i != j:
                cantidad_asignada = random.randint(0, capacidad_restante)
                asignacion[i][j] = cantidad_asignada
                capacidad_restante -= cantidad_asignada
    return asignacion

In [67]:
# Búsqueda local: Encuentra una mejor solución vecina mediante intercambio de recursos
def busqueda_local(asignacion_actual):
    mejor_asignacion = asignacion_actual
    mejor_distancia = calcular_distancia(asignacion_actual)
    num_iteraciones_sin_mejora = 0
    max_iteraciones_sin_mejora = 100000

    while num_iteraciones_sin_mejora < max_iteraciones_sin_mejora:
        # Generamos dos pares de índices de forma aleatoria = dos aristas
        i, j = random.randint(0, num_almacenes - 1), random.randint(0, num_almacenes - 1)
        k, l = random.randint(0, num_almacenes - 1), random.randint(0, num_almacenes - 1)
        # Que no sean las mismas
        if i != k or j != l:
            vecina_asignacion = [fila[:] for fila in asignacion_actual]
            cantidad_ij = min(vecina_asignacion[i][j], capacidades[k] - vecina_asignacion[k][l])
            cantidad_kl = min(vecina_asignacion[k][l], capacidades[i] - vecina_asignacion[i][j])

            # Realizamos un intercambio de valores entre dos asignaciones/aristas = Permutacion
            
            vecina_asignacion[i][j] -= cantidad_ij
            vecina_asignacion[k][l] += cantidad_ij
            vecina_asignacion[k][l] -= cantidad_kl
            vecina_asignacion[i][j] += cantidad_kl

            distancia_vecina = calcular_distancia(vecina_asignacion)
            
            # Si encontramos una solución mejor reseteamos el bucle, si no nos acercamos
            # al criterio de parada
            if distancia_vecina < mejor_distancia:
                mejor_asignacion = vecina_asignacion
                mejor_distancia = distancia_vecina
                num_iteraciones_sin_mejora = 0
            else:
                num_iteraciones_sin_mejora += 1

    return mejor_asignacion, mejor_distancia

# Ejecutar búsqueda local
solucion_inicial = generar_solucion_inicial()
mejor_solucion, mejor_distancia = busqueda_local(solucion_inicial)

# Mostrar resultados
print('Mejor Asignación Encontrada:')
for fila in mejor_solucion:
    print(fila)
print('Distancia Total de la Mejor Asignación:', mejor_distancia)


Mejor Asignación Encontrada:
[0, 10, 66, 16, 3]
[149, 0, 0, 0, 1]
[21, 103, 0, 55, 18]
[72, 65, 106, 0, 3]
[0, 18, 5, 15, 262]
Distancia Total de la Mejor Asignación: 12265


### -- SWARM INTELLIGENCE --

### IMPLEMENTACIÓN 2

In [ ]:
import numpy as np
import random

# Datos del problema (ajusta estos datos según tu instancia específica)
num_almacenes = 5
capacidades = [100, 150, 200, 250, 300]
distancias = [
    [0, 10, 20, 30, 40],
    [10, 0, 15, 25, 35],
    [20, 15, 0, 10, 20],
    [30, 25, 10, 0, 15],
    [40, 35, 20, 15, 0]
]

In [ ]:
# Función objetivo: Calcula la distancia total de la asignación actual
def calcular_distancia(asignacion):
    distancia_total = 0
    for i in range(num_almacenes):
        for j in range(num_almacenes):
            distancia_total += distancias[i][j] * asignacion[i][j]
    return distancia_total

# Genera una solución inicial aleatoria
def generar_solucion_inicial():
    asignacion = [[0] * num_almacenes for _ in range(num_almacenes)]
    for i in range(num_almacenes):
        capacidad_restante = capacidades[i]
        for j in range(num_almacenes):
            if i != j:
                cantidad_asignada = random.randint(0, capacidad_restante)
                asignacion[i][j] = cantidad_asignada
                capacidad_restante -= cantidad_asignada
    return asignacion

In [112]:
# Parámetros del algoritmo PSO
num_particulas = 50
max_iteraciones = 500
inercia = 0.2
cognitivo = 1.5
social = 1.5



# Inicializa el enjambre de partículas
enjambre = []
mejor_solucion_global = None
mejor_distancia_global = float('inf')

# Generamos el enjambre
# Generamos todas las partículas con su correspondiente asignación y fitness asociados
# Comparamos todas ellas y guardamos la mejor solución global

for _ in range(num_particulas):
    asignacion_particula = generar_solucion_inicial()
    distancia_particula = calcular_distancia(asignacion_particula)
    mejor_posicion_particula = asignacion_particula[:]
    mejor_distancia_particula = distancia_particula

    if distancia_particula < mejor_distancia_global:
        mejor_solucion_global = asignacion_particula[:]
        mejor_distancia_global = distancia_particula

    enjambre.append({
        'posicion': asignacion_particula,
        'distancia': distancia_particula,
        'mejor_posicion': mejor_posicion_particula,
        'mejor_distancia': mejor_distancia_particula
    })

# Algoritmo PSO
for iteracion in range(max_iteraciones):
    # Iteramos sobre todas las patículas del enjambre
    for particula in enjambre:
        # Generamos dos valores aleatorios 0 - 1
        r1, r2 = random.random(), random.random()
        # Iteramos sobre la matriz de asignaciones
        for i in range(num_almacenes):
            for j in range(num_almacenes):
                # Calculamos la velocidad de la partícula en cada asignación = combinación de inercia, cognitivo y social
                # Inercia = Efecto del estado anterior
                # Cognitivo = Influencia de la mejor solución de la partícula (STM)
                # Social = Influencia de la mejor solución global (LTM)
                velocidad = inercia * particula['posicion'][i][j] + \
                            cognitivo * r1 * (particula['mejor_posicion'][i][j] - particula['posicion'][i][j]) + \
                            social * r2 * (mejor_solucion_global[i][j] - particula['posicion'][i][j])
                # Definimos una nueva asignación de materiales en función del estado anterior y la velocidad
                nueva_asignacion = max(0, min(capacidades[i], particula['posicion'][i][j] + round(velocidad)))
                particula['posicion'][i][j] = nueva_asignacion
            
        # Calculamos el nuevo fitness de la partícula

        distancia_particula = calcular_distancia(particula['posicion'])

        # Actualizamos los valores de mejor posición de partícula y global
        if distancia_particula < particula['mejor_distancia']:
            particula['mejor_posicion'] = particula['posicion'][:]
            particula['mejor_distancia'] = distancia_particula

            if distancia_particula < mejor_distancia_global:
                mejor_solucion_global = particula['posicion'][:]
                mejor_distancia_global = distancia_particula

# Mostrar resultados
print('Mejor Asignación Encontrada:')
for fila in mejor_solucion_global:
    print(fila)
print('Distancia Total de la Mejor Asignación:', mejor_distancia_global)


Mejor Asignación Encontrada:
[0, 100, 2, 100, 100]
[150, 0, 150, 150, 0]
[200, 200, 0, 200, 200]
[250, 250, 250, 0, 250]
[300, 300, 300, 300, 0]
Distancia Total de la Mejor Asignación: 17695
